In [1]:
# Define the directory structure using file.path()
my_dir <- file.path("C:/Users/GPEL_HPENVY/Desktop/Project_Predictive_Analytics_R")

# Check if the directory exists
if (dir.exists(my_dir)) {
  # Set the working directory
  setwd(my_dir)
  cat("Working directory set to:", getwd(), "\n")
} else {
  cat("Directory does not exist:", my_dir, "\n")
}

Working directory set to: C:/Users/GPEL_HPENVY/Desktop/Project_Predictive_Analytics_R 


In [2]:
# set file path
# read csv file
#setwd('H:/GPEL_HPENVY/Documents/UNSW/07_ZZBU6511/Assessment3/')
employees <- read.csv(file = 'employees2.csv')

# Modify the column names to remove unwanted characters (assuming the problematic column is named "ï..Age")
colnames(employees) <- gsub("ï\\.{2}", "", colnames(employees))

In [3]:
# Inspect file
head(employees)


Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,JobInvolvement,JobRole,...,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,HighPerformance
26,No,Travel_Frequently,Research & Development,10,3,3,Male,4,Laboratory Technician,...,1,1,8,3,3,7,7,7,7,0
33,No,Travel_Rarely,Research & Development,13,5,4,Male,3,Healthcare Representative,...,1,0,14,6,3,0,0,0,0,0
59,No,Travel_Rarely,Research & Development,5,3,3,Female,4,Laboratory Technician,...,1,3,12,3,2,1,0,0,0,1
42,No,Travel_Rarely,Research & Development,21,3,2,Male,4,Healthcare Representative,...,3,2,10,2,2,0,0,0,0,0
24,No,Travel_Rarely,Research & Development,27,1,4,Female,2,Manufacturing Director,...,2,2,5,6,3,4,2,3,2,0
29,No,Travel_Rarely,Research & Development,37,2,3,Male,4,Research Scientist,...,3,1,10,3,3,10,7,0,7,1


In [4]:
# Add new binary variables to the employees data frame
employees$Is.Attrition = ifelse(employees$Attrition == "Yes", 1, 0)

In [5]:
# Split the data to training and testing
set.seed(57)
rows <- sample(nrow(employees), 0.8*nrow(employees))
training <- employees[rows, ]
testing <- employees[-rows, ]

In [6]:
# Making the predictions.  Don't include Over18 since all is Yes. Use upto 2* statistical significant.
logmod <- glm(
  formula = Is.Attrition ~ BusinessTravel + OverTime + JobRole +
    DistanceFromHome + JobSatisfaction + EnvironmentSatisfaction + JobInvolvement + 
     NumCompaniesWorked + WorkLifeBalance,
  data = training,
  family = binomial()
)
# Print summary of the prediction
summary(logmod)


Call:
glm(formula = Is.Attrition ~ BusinessTravel + OverTime + JobRole + 
    DistanceFromHome + JobSatisfaction + EnvironmentSatisfaction + 
    JobInvolvement + NumCompaniesWorked + WorkLifeBalance, family = binomial(), 
    data = training)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6482  -0.5358  -0.2931  -0.1270   3.0825  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     -2.093375   1.365246  -1.533 0.125194    
BusinessTravelTravel_Frequently  1.971818   0.558940   3.528 0.000419 ***
BusinessTravelTravel_Rarely      1.181131   0.523782   2.255 0.024133 *  
OverTimeYes                      1.824873   0.248857   7.333 2.25e-13 ***
JobRoleHuman Resources           3.410067   1.154509   2.954 0.003140 ** 
JobRoleLaboratory Technician     3.839303   1.081425   3.550 0.000385 ***
JobRoleManager                   2.009682   1.182956   1.699 0.089345 .  
JobRoleManufacturing Director    2

In [7]:
# Do prediction using the training parameters
# Get the probabilities from the model and add them into a new column
training$Probability <- predict(
  object = logmod,
  newdata = training,
  # To get probabilities we set the type to "response"
  type = "response"
)

In [8]:
# Compute ROC and Printing the area under the ROC curve
roc(
  formula = training$Is.Attrition ~ training$Probability,
  auc = TRUE
)

ERROR: Error in roc(formula = training$Is.Attrition ~ training$Probability, auc = TRUE): could not find function "roc"


In [ ]:
# Turn the probabilities into predictions
training$Prediction = ifelse(training$Probability >= 0.2, 1, 0)

In [ ]:
# Get the confusion matrix
conf_matrix_train <- table(training$Prediction, training$Is.Attrition)

# Show the confusion matrix
conf_matrix_train


In [ ]:
# Calculate and show the accuracy
accuracy <- sum(diag(conf_matrix_train))/sum(conf_matrix_train)
message("Accuracy: ", accuracy)

# Calculate and show the sensitivity
sensitivity <- conf_matrix_train[2,2]/sum(conf_matrix_train[,2])
message("Sensitivity: ", sensitivity)

# Calculate and show the specificity
specificity <- conf_matrix_train[1,1]/sum(conf_matrix_train[,1])
message("Specificity: ", specificity)

In [ ]:
# Testing the model - using the testing dataset and using the Prediction models
testing$ProbabilityTest <- predict(logmod, newdata = testing, type = "response")
testing$PredictionTest <- ifelse(testing$ProbabilityTest >= 0.2, 1, 0)
conf_matrix_test <- table(testing$PredictionTest, testing$Is.Attrition)
message("Confusion matrix for model 1:")
conf_matrix_test

In [ ]:
# Calculate and show the accuracy
accuracy <- sum(diag(conf_matrix_test))/sum(conf_matrix_test)
message("Accuracy: ", accuracy)

# Calculate and show the sensitivity
sensitivity <- conf_matrix_test[2,2]/sum(conf_matrix_test[,2])
message("Sensitivity: ", sensitivity)

# Calculate and show the specificity
specificity <- conf_matrix_test[1,1]/sum(conf_matrix_test[,1])
message("Specificity: ", specificity)
